### Info

Author: Jelle Poland - jellepoland@gmail.com \
Citing: https://doi.org/10.3390/en16145264 \
License: ... \
Github: ...


### Initialisation


In [1]:
# making things autorelad - needed for Jupyter Kernel
%load_ext autoreload
%autoreload 2
%matplotlib widget

# Define the right-path
import sys
import os
folder_path = '../'
os.chdir(folder_path)
sys.path.append(os.getcwd())

# Importing modules
from src.initialisation import load_surfplan, pulley_connectivity
from src.initialisation.input_classes import input_VSM , input_bridle_aero, input_structural_solver
from src.initialisation import input_particleSystem,particles_with_rotational_resistance
from src.particleSystem.ParticleSystem import ParticleSystem
from src.coupling import coupling_aero2struc, coupling_struc2aero
from src.structural import structural_model, structural_mesher
from src.solver import solver_main, solver_utils
from src.post_processing import functions_print, functions_plot, post_processing_utils, post_processing_main
from src.initialisation import actuation_relations
from src.aerodynamic import VSM, breukels_2D, plate_aero, bridle_line_system_aero, tether_aero
from src.initialisation.yaml_loader import config

# from test import test_main
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import time
import scipy.optimize
import yaml
import importlib
import pytest
import pandas as pd
import dill
from IPython.display import display, Latex # for pretty printing


## Mutable variables
# Initializing Mutable Variables
points = config.kite.points_ini
# defining vel_app (vector of vel_app_norm)
vel_app = config.vel_wind - config.vel_kite

# Should be the same for each kite
(
    connectivity_matrix,
    wing_connectivity,
) = input_particleSystem.define_connectivity_matrix(config)
params_dict = input_particleSystem.define_params(
    config, wing_connectivity, connectivity_matrix
)
initial_conditions = input_particleSystem.define_initial_conditions_kite(
    config)
points_between_dict = particles_with_rotational_resistance.extract_points_between_dict(
    config
)
if config.is_with_initial_plot:
    functions_plot.plot_initial_geometry(config, points_between_dict)

is_with_rotational_resistance = False
if config.kite_name == "V9_60C":
    is_with_rotational_resistance = True

if is_with_rotational_resistance:
    (
        leadingedge_rotational_resistance_dict,
        strut_rotational_resistance_dict,
    ) = particles_with_rotational_resistance.extract_rotational_resistances_dicts(
        points_between_dict, config
    )
    # first do the struts
    k_bend_strut = 1e10
    params_dict = particles_with_rotational_resistance.initialize_bending_spring(
        k_bend_strut,
        initial_conditions,
        params_dict,
        connectivity_matrix,
        strut_rotational_resistance_dict,
    )
    # secondly do the leading-edge
    k_bend_leadingedge = 1e4
    params_dict = particles_with_rotational_resistance.initialize_bending_spring(
        k_bend_leadingedge,
        initial_conditions,
        params_dict,
        connectivity_matrix,
        leadingedge_rotational_resistance_dict,
    )
# Should be the same for each kite
psystem = ParticleSystem(connectivity_matrix, initial_conditions, params_dict)

### AeroStructural Simulation

    1. run staight_symmetric
    2. run steady_circular


In [2]:
# ## straight symmetric case
# sim_name = "straight_symmetric"
# points, print_data, plot_data, animation_data = solver_main.run_aerostructural_solver(
#     points,
#     vel_app,
#     psystem,
#     params_dict,
#     config,
#     input_VSM,
#     input_bridle_aero,
#     is_with_vk_optimization=False,
#     is_circular_case=False,
#     is_run_only_1_time_step=False,
#     is_print_intermediate_results=True,
#     is_with_gravity=True,
#     sim_name="straight_symmetric",
#     is_with_velocity_initialization=False,
# )

# TODO:  Data-saving procedure for the torque paper, should be stream-lined
# data_names = [
#     "points",
#     "psystem",
#     "print_data",
#     "plot_data",
#     "animation_data",
#     "config",
#     "vel_app",
#     "sim_name",
#     "input_VSM",
# ]
# loaded_data = {}
# for name in data_names:
#     with open(
#         f"results/{config.kite_name}/torque_paper/2023_26_14_straight/{name}.pkl", "rb"
#     ) as f:
#         loaded_data[name] = dill.load(f)

# points = loaded_data["points"]
# psystem = loaded_data["psystem"]
# print_data = loaded_data["print_data"]
# plot_data = loaded_data["plot_data"]
# animation_data = loaded_data["animation_data"]
# config = loaded_data["config"]
# vel_app = loaded_data["vel_app"]
# sim_name = loaded_data["sim_name"]
# input_VSM = loaded_data["input_VSM"]
# folder_name = f"results/{config.kite_name}/torque_paper/2023_26_14_v1/"


# circular case
sim_name = "steady_circular"
points, print_data, plot_data, animation_data = solver_main.run_aerostructural_solver(
    points,
    vel_app,
    psystem,
    params_dict,
    config,
    input_VSM,
    input_bridle_aero,
)

# Output settings
is_with_printing = True
is_with_plotting = True
is_with_animation = False
is_with_save = False

if is_with_printing:
    post_processing_main.print_results(
        points,
        print_data,
        config,
    )
if is_with_plotting:
    post_processing_main.plot(
        plot_data,
        points,
        vel_app,
        config,
    )
if is_with_animation:
    post_processing_main.animate(
        animation_data,
        vel_app,
        config,
        input_VSM,
    )
# if is_with_save:
#     to_be_saved_data = [
#         [points, "points"],
#         [psystem, "psystem"],
#         [print_data, "print_data"],
#         [plot_data, "plot_data"],
#         [animation_data, "animation_data"],
#         [config, "config"],
#         [vel_app, "vel_app"],
#         [sim_name, "sim_name"],
#         [input_VSM, "input_VSM"],
#     ]

#     # Ensure the folder exists
#     if not os.path.exists(folder_name):
#         os.makedirs(folder_name)

#     # Serialize and save all data with dill
#     for item in to_be_saved_data:
#         data, name = item
#         with open(f"{folder_name}{name}.pkl", "wb") as f:
#             dill.dump(data, f)

Initial depower tape length: 1.098m
Desired depower tape length: 1.108m
Initial steering tape length right: 1.601m
Desired steering tape length right: 1.401m
 
Running aero-structural simulation
----------------------------------- 
i:0, t-step:0.01, residual_f: 586.868N (aero: 0.336s, struc: 0.040s)
i:1, t-step:0.02, residual_f: 2902.705N (aero: 0.471s, struc: 0.051s)
i:2, t-step:0.03, residual_f: 3246.574N (aero: 0.480s, struc: 0.035s)
i:3, t-step:0.04, residual_f: 3078.110N (aero: 0.441s, struc: 0.016s)
i:4, t-step:0.05, residual_f: 2978.803N (aero: 0.436s, struc: 0.017s)
i:5, t-step:0.06, residual_f: 2680.234N (aero: 0.441s, struc: 0.015s)
i:6, t-step:0.07, residual_f: 2428.547N (aero: 0.438s, struc: 0.012s)
i:7, t-step:0.08, residual_f: 2123.265N (aero: 0.434s, struc: 0.021s)
i:8, t-step:0.09, residual_f: 1871.036N (aero: 0.458s, struc: 0.038s)
i:9, t-step:0.10, residual_f: 562.688N (aero: 0.465s, struc: 0.023s)
i:10, t-step:0.11, residual_f: 420.154N (aero: 0.465s, struc: 0.028s)


KeyboardInterrupt: 

# Simulations In Order


### 1. static-aero


In [ ]:
# external force
# Struc --> aero
points_left_to_right = coupling_struc2aero.order_struc_nodes_right_to_left(
    points, config.kite.connectivity.plate_point_indices
)
# Wing Aerodynamic
(
    force_aero_wing_VSM,
    moment_aero_wing_VSM,
    F_rel,
    ringvec,
    controlpoints,
    wingpanels,
    rings,
    coord_L,
    coord_refined,
) = VSM.calculate_force_aero_wing_VSM(points_left_to_right, vel_app, input_VSM)
# Aero --> struc
force_aero_wing = coupling_aero2struc.aero2struc(
    points,
    config.kite.connectivity.wing_ci,
    config.kite.connectivity.wing_cj,
    config.kite.connectivity.plate_point_indices,
    force_aero_wing_VSM,
    moment_aero_wing_VSM,
    ringvec,
    controlpoints,
)  # Put in the new positions of the points

# Bridle Aerodynamics
if config.is_with_aero_bridle:
    force_aero_bridle = bridle_line_system_aero.calculate_force_aero_bridle_thedens2022(
        points, vel_app, input_bridle_aero
    )
else:
    force_aero_bridle = [0]
force_aero = force_aero_wing + force_aero_bridle

functions_print.print_aero(points, vel_app, force_aero_wing, force_aero_bridle, config)

#######
points_wing = np.copy(points)
evaluation_point = np.array([max(points_wing[:, 0]) / 2, 0, max(points_wing[:, 2])])
evaluation_point = np.zeros(3)
for i, point in enumerate(points_wing):
    points_wing[i] = point - evaluation_point
    if i not in config.kite.connectivity.plate_point_indices:
        points_wing[i] = np.array([0, 0, 0])
        force_aero[i] = np.array([0, 0, 0])

moments_wing = np.cross(points_wing, force_aero)
print(
    f"Moments Full Wing on mid-span, mid-chord point --> Mx: {moments_wing[:,0].sum():.3f} My: {moments_wing[:,1].sum():.3f} Mz: {moments_wing[:,2].sum():.2f}"
)
for i, point in enumerate(points_wing):
    if points_wing[i][1] > 0:  # taking only the right wing
        points_wing[i] = np.array([0, 0, 0])
        force_aero[i] = np.array([0, 0, 0])

moments_half_wing = np.cross(points_wing, force_aero)
print(
    f"Moments Half Right Wing on mid-span, mid-chord point --> Mx: {moments_half_wing[:,0].sum():.3f} My: {moments_half_wing[:,1].sum():.3f} Mz: {moments_half_wing[:,2].sum():.2f}"
)
print(f"norm of Va: {np.linalg.norm(vel_app):.2f}m/s")
#######

# plotting
post_processing_main.plot(
    [
        [wingpanels, controlpoints, rings, coord_L, F_rel],
        config.kite.wing_rest_lengths_initial,
        config.kite.bridle_rest_lengths_initial,
    ],
    points,
    vel_app,
    config,
)

In [ ]:
from scipy.spatial import ConvexHull

wing_nodes = []
for i in wing_connectivity:
    wing_nodes.append(psystem.x_current_2D[i[0]])
    wing_nodes.append(psystem.x_current_2D[i[1]])
wing_nodes = np.array(wing_nodes)


def calculate_projected_area(points):
    # Project points onto the x,y plane
    xy_points = points[:, :2]

    # Find the convex hull
    hull = ConvexHull(xy_points)
    hull_points = xy_points[hull.vertices]

    # Using the shoelace formula
    x = hull_points[:, 0]
    y = hull_points[:, 1]

    print("hull.area", hull.area)
    return 0.5 * np.abs(np.dot(x, np.roll(y, 1)) - np.dot(y, np.roll(x, 1)))


span = max(wing_nodes[:, 1]) - min(wing_nodes[:, 1])
print("span*chord:", span * config.kite.ref_chord)

# print(ConvexHull(wing_nodes).area)
# Example usage
print("Maximum Projected Area:", calculate_projected_area(wing_nodes))
print("Maximum Projected Area:", calculate_projected_area(points))
# print(wing_nodes)

### 1. straight_symmetric - reel-in


In [ ]:
sim_name = "straight_symmetric"
is_with_vk_optimization = False
is_circular_case = False
is_run_only_1_time_step = False
is_print_intermediate_results = False
is_with_gravity = False
is_with_velocity_initialization = True

is_with_printing = True
is_with_plotting = True
is_with_animation = False

points, print_data, plot_data, animation_data = solver_main.run_aerostructural_solver(
    points,
    vel_app,
    psystem,
    params_dict,
    config,
    input_VSM,
    input_bridle_aero,
    is_with_vk_optimization,
    is_circular_case,
    is_run_only_1_time_step,
    is_print_intermediate_results,
    is_with_gravity,
)
if is_with_printing:
    post_processing_main.print_results(
        points,
        print_data,
        config,
    )
if is_with_plotting:
    post_processing_main.plot(
        plot_data,
        points,
        vel_app,
        config,
    )
if is_with_animation:
    post_processing_main.animate(
        animation_data,
        vel_app,
        config,
        input_VSM,
    )
np.save(
    f"results/{config.kite_name}/points/up_{str(int(100*config.u_p))}_{sim_name}.npy",
    points,
)

### 2. crosswind_symmetric - reel-out


In [ ]:
sim_name = "perfect_crosswind"
is_with_vk_optimization = True
is_circular_case = False
is_run_only_1_time_step = False
is_print_intermediate_results = False
is_with_gravity = False
is_with_velocity_initialization = False

is_with_printing = True
is_with_plotting = True
is_with_animation = False

points, print_data, plot_data, animation_data = solver_main.run_aerostructural_solver(
    points,
    vel_app,
    psystem,
    params_dict,
    config,
    input_VSM,
    input_bridle_aero,
    is_with_vk_optimization,
    is_circular_case,
    is_run_only_1_time_step,
    is_print_intermediate_results,
    is_with_gravity,
)
if is_with_printing:
    post_processing_main.print_results(
        points,
        print_data,
        config,
    )
if is_with_plotting:
    post_processing_main.plot(
        plot_data,
        points,
        vel_app,
        config,
    )
if is_with_animation:
    post_processing_main.animate(
        animation_data,
        vel_app,
        config,
        input_VSM,
    )
np.save(
    f"results/{config.kite_name}/points/up_{str(int(100*config.u_p))}_{sim_name}.npy",
    points,
)

### 3. steady_circular - turning


In [ ]:
sim_name = "steady_circular"
is_with_vk_optimization = True
is_run_only_1_time_step = False
is_circular_case = True
is_print_intermediate_results = True
is_with_gravity = False
is_with_velocity_initialization = False

is_with_printing = True
is_with_plotting = True
is_with_animation = False

points, print_data, plot_data, animation_data = solver_main.run_aerostructural_solver(
    points,
    vel_app,
    psystem,
    params_dict,
    config,
    input_VSM,
    input_bridle_aero,
    is_with_vk_optimization,
    is_circular_case,
    is_run_only_1_time_step,
    is_print_intermediate_results,
    is_with_gravity,
)
if is_with_printing:
    post_processing_main.print_results(
        points,
        print_data,
        config,
    )
if is_with_plotting:
    post_processing_main.plot(
        plot_data,
        points,
        vel_app,
        config,
    )
if is_with_animation:
    post_processing_main.animate(
        animation_data,
        vel_app,
        config,
        input_VSM,
    )
np.save(
    f"results/{config.kite_name}/points/up_{str(int(100*config.u_p))}_{sim_name}.npy",
    points,
)

In [ ]:
# TODO: remove, this is the code to save all the simulation data
folder_name = f"results/{config.kite_name}/torque_paper/2023_26_12_v2/"
to_be_saved_data = [
    [points, "points"],
    [print_data, "print_data"],
    [plot_data, "plot_data"],
    [animation_data, "animation_data"],
    [config, "config"],
    [vel_app, "vel_app"],
    [sim_name, "sim_name"],
    [input_VSM, "input_VSM"],
]

# Ensure the folder exists
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Serialize and save all data with dill
for item in to_be_saved_data:
    data, name = item
    with open(f"{folder_name}{name}.pkl", "wb") as f:
        dill.dump(data, f)

In [ ]:
post_processing_main.animate(
    animation_data,
    vel_app,
    config,
    input_VSM,
)

# Other


## Plotting Solution Space for Kite Velocity Optimization


In [ ]:
# Loading points from 1.
points = np.load(
    f"results/{config.kite_name}/points/up_{str(int(100*config.u_p))}_straight_symmetric.npy"
)

# 2.1 Plotting Solution Space
is_with_plot_crosswind_vk_solution_space = True
if is_with_plot_crosswind_vk_solution_space:
    vel_kite_x_values = []
    fx_values = []
    aoa_values = []

    # getting the angle of mid-chord
    points_LE_segment = points[
        config.kite.connectivity.plate_point_indices[int(
            config.kite.n_segments / 2)]
    ][:2]
    points_TE_segment = points[
        config.kite.connectivity.plate_point_indices[int(
            config.kite.n_segments / 2)]
    ][2:]
    LE_point_diff = points_LE_segment[1] - points_LE_segment[0]
    TE_point_diff = points_TE_segment[1] - points_TE_segment[0]
    mid_point_LE = points_LE_segment[0] + 0.5 * LE_point_diff
    mid_point_TE = points_TE_segment[0] + 0.5 * TE_point_diff
    trim_angle = np.arctan(mid_point_LE[0] / mid_point_LE[2])
    delta_z_chord = mid_point_LE[2] - mid_point_TE[2]
    delta_x_chord = np.abs(mid_point_LE[0] - mid_point_TE[0])
    angle_of_mid_chord = np.rad2deg(np.arctan(delta_z_chord / delta_x_chord))

    # TODO: make this a function in a separate file
    for vel_kite_x_old in np.arange(10, 40, 3):
        vel_app[0] = float(config.vel_wind[0] + vel_kite_x_old)
        fx_sum = solver_utils.calculate_fx(
            -vel_kite_x_old, vel_app, points, config, input_VSM, input_bridle_aero
        )

        # Append data to arrays for plotting
        vel_kite_x_values.append(-vel_kite_x_old)
        fx_values.append(fx_sum)

        # Also getting the aoa for plotting
        angle_va_with_xy = np.rad2deg(np.arctan(vel_app[2] / vel_app[0]))
        aoa = angle_of_mid_chord + angle_va_with_xy
        aoa_values.append(aoa)

        # print(f'vel_kite_x: {-vel_kite_x_old}, fx: {fx_sum}')

    fig, (ax1, ax2, ax3) = plt.subplots(
        3, 1, figsize=(8, 15)
    )  # Adjust the figsize as needed

    # Plot the first data in the first subplot
    ax1.plot(fx_values, vel_kite_x_values, marker="o", linestyle="--")
    ax1.set_title("vel_kite_x vs fx")
    ax1.set_xlabel("fx")
    ax1.set_ylabel("vel_kite_x")
    ax1.grid(True)
    # ax1.set_xlim(-40, -10)

    # create the second y-axis
    ax1_twin = ax1.twinx()

    # Plot the second data on the right y-axis
    ax1_twin.plot(
        fx_values, aoa_values, marker="s", linestyle="--", color="r", label="aoa"
    )
    ax1_twin.set_ylabel("aoa", color="r")

    # Add legend
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax1_twin.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc="upper left")

    # Plot the second data in the second subplot
    ax2.plot(
        vel_kite_x_values, fx_values, marker="o", color="b", linestyle="--", label="aoa"
    )  # Modify these with your data
    ax2.set_title("vel_kite_x vs fx")
    ax2.set_xlabel("vel_kite_x")
    ax2.set_ylabel("fx")
    ax2.grid(True)
    # ax2.set_xlim(-40, -5)  # Modify the x-axis limits as needed

    ax3.plot(
        aoa_values, fx_values, marker="o", color="r", linestyle="--", label="aoa"
    )  # Modify these with your data
    ax3.set_title("aoa vs fx")
    ax3.set_xlabel("aoa")
    ax3.set_ylabel("fx")
    ax3.grid(True)

    plt.tight_layout()  # Ensures proper spacing between subplots
    plt.show()

## Old files for circular flight analysis


### Old printing out statements


In [ ]:
# # Printing out results
# force_aero = print_data[8][0]
# force_aero_sum = np.sum(force_aero, axis=0)
# print(f"sum(Fa):{force_aero_sum}N")

# moment_aero = np.cross(points, force_aero)
# moment_aero_sum = np.sum(moment_aero, axis=0)
# print(f"sum(Ma):{moment_aero_sum}Nm")

# center_of_fa = solver_utils.calculate_center_of_gravity(points, force_aero)
# print(f"Force center: {center_of_fa}m")
# print(f"Ma at force-center:{np.cross(center_of_fa,force_aero_sum)[0]:.0f}Nm")

# roll_moment = np.sum(np.cross(points, force_aero), axis=0)[0]
# print(f"Roll Moment: {roll_moment:.0f} Nm")
# print(f"Ma_roll due to to Fa_side: {force_aero_sum[1]*center_of_fa[2]:.0f}Nm")
# print(
#     f"Ma[0] {np.sum(moment_aero[:,0]):.0f}Nm, Ma[1] {np.sum(moment_aero[:,1]):.0f}Nm, Ma[2] {np.sum(moment_aero[:,2]):.0f}Nm"
# )

# print(f"Ma individual: {solver_utils.calculate_moment(points,force_aero)}")

# print(f"")
# print(f"Equal forces, set Fc = Fa_side")
# print(f"--------------------------")
# (
#     force_centrifugal_i_array,
#     r_0,
#     v_k_i_array,
#     center_of_gravity,
# ) = solver_utils.calculate_force_centrifugal_distribution(
#     force_aero, config.kite.mass_points, vel_app, points
# )
# force_centrifugal = np.array(
#     [np.array([0, 1, 0]) * force for i, force in enumerate(force_centrifugal_i_array)]
# )
# moment_centrifugal = np.cross(points, force_centrifugal)
# print(
#     f"Fa_side: {np.sum(force_aero[:, 1]):.0f}N, Fc: {np.sum(force_centrifugal_i_array):.0f}N, diff: {np.abs(np.sum(force_centrifugal_i_array) + (np.sum(force_aero[:,1]))):.0f}N"
# )
# print(
#     f"Ma_side: {np.sum(moment_aero[:, 0]):.0f}Nm, Mc: {np.sum(moment_centrifugal[:, 0]):.0f}Nm, diff: {np.abs(np.sum(moment_centrifugal[:, 0]) + (np.sum(moment_aero[:,0]))):.0f}Nm, %: {100* np.abs(np.sum(moment_centrifugal[:, 0]) + (np.sum(moment_aero[:,0])))/np.sum(moment_aero[:, 0]):.3f}"
# )
# print(f"r_0: {r_0:.2f}m")
# print(
#     f"Ma at ac:{np.sum(force_aero[:, 1])*center_of_fa[2]:.0f}Nm, {np.cross(center_of_fa,[0,np.sum(force_aero[:, 1]),0])[0]:.0f}Nm"
# )
# print(
#     f"Mc at cg:{np.sum(force_centrifugal_i_array)*center_of_gravity[2]:.0f}Nm, {np.cross(center_of_gravity,[0,np.sum(force_centrifugal_i_array),0])[0]:.0f}Nm"
# )
# print(f"cg: {center_of_gravity}m")

# print(f"")
# print(f"Equal Moments, set Mc = Ma_side")
# print(f"--------------------------")
# (
#     force_centrifugal_i_array,
#     r_0,
#     v_k_i_array,
#     center_of_gravity,
# ) = solver_utils.calculate_force_centrifugal_distribution_moment_based(
#     moment_aero, config.kite.mass_points, vel_app, points
# )
# force_centrifugal = np.array(
#     [np.array([0, 1, 0]) * force for i, force in enumerate(force_centrifugal_i_array)]
# )
# moment_centrifugal = np.cross(points, force_centrifugal)
# print(
#     f"Fa_side: {np.sum(force_aero[:, 1]):.0f}N, Fc: {np.sum(force_centrifugal_i_array):.0f}N, diff: {np.abs(np.sum(force_centrifugal_i_array) + (np.sum(force_aero[:,1]))):.0f}N"
# )
# print(
#     f"Ma_side: {np.sum(moment_aero[:, 0]):.0f}Nm, Mc: {np.sum(moment_centrifugal[:, 0]):.0f}Nm, diff: {np.abs(np.sum(moment_centrifugal[:, 0]) + (np.sum(moment_aero[:,0]))):.0f}Nm, %: {100* np.abs(np.sum(moment_centrifugal[:, 0]) + (np.sum(moment_aero[:,0])))/np.sum(moment_aero[:, 0]):.3f}"
# )
# print(f"r_0: {r_0:.2f}m")

# print(f"")
# print(f"Tether Tension, set Fc = Fa_side + F_tether_side")
# print(f"--------------------------")

r_0_previous = 100.0
length_tether = 200.0
is_print_intermediate_results = True
(
    force_centrifugal,
    r_0,
    v_k_i_array,
    center_of_gravity,
) = solver_utils.calculate_force_centrifugal_distribution_with_tether_tension(
    r_0_previous,
    force_aero,
    config.kite.mass_points,
    vel_app,
    points,
    length_tether,
    is_print_intermediate_results,
)
# print(f"Fc: {np.sum(force_centrifugal,axis=0)}N")
# moment_centrifugal = np.cross(points, force_centrifugal)
# print(
#     f"Fa_side: {np.sum(force_aero[:, 1]):.0f}N, Fc: {np.sum(force_centrifugal_i_array):.0f}N, diff: {np.abs(np.sum(force_centrifugal_i_array) + (np.sum(force_aero[:,1]))):.0f}N"
# # )
# print(
#     f"Ma_side: {np.sum(moment_aero[:, 0]):.0f}Nm, Mc: {np.sum(moment_centrifugal[:, 0]):.0f}Nm, diff: {np.abs(np.sum(moment_centrifugal[:, 0]) + (np.sum(moment_aero[:,0]))):.0f}Nm, %: {100* np.abs(np.sum(moment_centrifugal[:, 0]) + (np.sum(moment_aero[:,0])))/np.sum(moment_aero[:, 0]):.3f}"
# )
# print(f"r_0: {r_0:.2f}m")


# TODO: should take spanwise variation of Va into account for aero model
# calculate new apparant wind speed
# vel_app_array = [(config.vel_wind - vel_kite_i_array[i]) for i,point in enumerate(points)]


# print(f" ")
# print(f" --- turning variable initialiasiation --- ")
# print(f"Distance to turning center: r_0: {r_0:.2f}m")
# # print(f'angular velocity: {angular_velocity_cg:.2f}rad/s')
# # print(f'center of gravity: {center_of_gravity}, r_i_cg: {center_of_gravity[1]:.2f}m')
# print(
#     f"Diff Fy: {np.sum(force_centrifugal_i_array) - (-np.sum(force_aero[:,1])) :.3f}N, (Fc: {np.sum(force_centrifugal_i_array):.2f}N, Fa,y: {np.sum(force_aero[:,1]):.2f}N)"
# )

### Old (pre-alex) circular aerostructural loop


In [ ]:
# initialisation
functions_print.print_settings(vel_app, config)
it_check_turning = 1  # number of iterations between printing out results
it_delta_ls_update = 2  # every 5 iterations, increase delta_ls
# it_update_vk = 2 #numer of iterations at which Vk needs updating
tol_fx_ratio_to_fz = 0.05  # tolerance for fx, if fx>tol*fz, then update vk_x
r_0_new = r_0
# tolerance for convergence
tol_r_0 = 0.25
tol_points_turning = 0.05
tol_solver_max_error = 0.5
# stopping conditions
max_points_diff_turning = 5  # if changes are larger than 5meters, then stop
max_sideangle = 5  # if more to the side than 5degrees, then stop
max_aerostructural_iter = 300

is_convergence = False
for it in range(max_aerostructural_iter + 1):
    # updating delta_ls, maxing it now at .2
    if it % it_delta_ls_update == 0 and it < 20:
        delta_ls = 0.005
        # spring[1] is the left steering tape
        bridle_rest_lengths[2] += delta_ls
        # spring[2] is the right steering tape
        bridle_rest_lengths[3] -= delta_ls

    print(f" ")
    # AERODYNAMICS
    time_iter = time.time()  # to print out each iteration time
    # Struc --> aero
    points_left_to_right = coupling_struc2aero.order_struc_nodes_right_to_left(
        points, config.kite.connectivity.plate_point_indices
    )
    # Wing Aerodynamic
    (
        force_aero_wing_VSM,
        moment_aero_wing_VSM,
        F_rel,
        ringvec,
        controlpoints,
        wingpanels,
        rings,
        coord_L,
        coord_refined,
    ) = VSM.calculate_force_aero_wing_VSM(points_left_to_right, vel_app, input_VSM)

    force_aero_wing = coupling_aero2struc.aero2struc(
        points,
        config.kite.connectivity.wing_ci,
        config.kite.connectivity.wing_cj,
        config.kite.connectivity.plate_point_indices,
        force_aero_wing_VSM,
        moment_aero_wing_VSM,
        ringvec,
        controlpoints,
    )  # Put in the new positions of the points
    # Bridle Aerodynamics
    if config.is_with_aero_bridle:
        force_aero_bridle = (
            bridle_line_system_aero.calculate_force_aero_bridle_thedens2022(
                points, vel_app, input_bridle_aero
            )
        )
    else:
        force_aero_bridle = [0]
    force_aero = force_aero_wing + force_aero_bridle

    # TODO: remove
    # calculating the moment of the aerodynamic forces
    moment_aero = np.cross(points, force_aero)
    # saving old force_aero for printing
    force_aero = force_aero
    # moment_aero = np.array([np.cross(point,force_aero[i]) for i,point in enumerate(points)])
    # Ma_x = np.array(force_aero[:, 2] * points[:, 1] - force_aero[:, 1] * points[:, 2])
    # Ma_y = np.array(force_aero[:, 0] * points[:, 2] - force_aero[:, 2] * points[:, 0])
    # Ma_z = np.array(force_aero[:, 1] * points[:, 0] - force_aero[:, 0] * points[:, 1])

    time_iter_aero = time.time()

    # Updating the velocity of the kite in the x-direction, to make Fx go to zero
    # run if fx > tol_fx_ratio_to_fz*Fz
    force_resultant_x = solver_utils.calculate_fx(
        -vel_app[0], vel_app, points, config, input_VSM, input_bridle_aero
    )

    if np.abs(force_resultant_x) > tol_fx_ratio_to_fz * np.abs(
        np.sum(force_aero[:, 2])
    ):
        print(f"i{it}, updating vel_kite_x")
        tol_vk_optimization = 1e-1
        vel_app[0] = -solver_utils.optimalisation_of_vk_for_fx_0(
            -vel_app[0],
            vel_app,
            points,
            config,
            input_VSM,
            input_bridle_aero,
            tol_vk_optimization,
        )

    # CALCULATING CENTRIFUGAL FORCE DISTRIBUTION
    # calculate centrifugal force distribution, i.e. the force on each node Fc,i
    (
        force_centrifugal_i_array,
        r_0_new,
        v_k_i_array,
        center_of_gravity,
    ) = solver_utils.calculate_force_centrifugal_distribution_moment_based(
        moment_aero, config.kite.mass_points, vel_app, points
    )
    fc_moment = np.sum(force_centrifugal_i_array)
    mc_moment = np.sum(
        np.cross(
            points,
            np.array(
                [
                    np.array([0, 1, 0]) * force
                    for i, force in enumerate(force_centrifugal_i_array)
                ]
            ),
        ),
        axis=0,
    )[0]
    (
        force_centrifugal_i_array,
        r_0_new,
        v_k_i_array,
        center_of_gravity,
    ) = solver_utils.calculate_force_centrifugal_distribution(
        force_aero, config.kite.mass_points, vel_app, points
    )
    fc_equal = np.sum(force_centrifugal_i_array)
    mc_equal = np.sum(
        np.cross(
            points,
            np.array(
                [
                    np.array([0, 1, 0]) * force
                    for i, force in enumerate(force_centrifugal_i_array)
                ]
            ),
        ),
        axis=0,
    )[0]
    (
        force_centrifugal_i_array,
        r_0_new,
        v_k_i_array,
        center_of_gravity,
    ) = solver_utils.calculate_force_centrifugal_distribution_with_tether_tension(
        r_0_new, force_aero, config.kite.mass_points, vel_app, points
    )
    fc_ft = np.sum(force_centrifugal_i_array)
    mc_ft = np.sum(
        np.cross(
            points,
            np.array(
                [
                    np.array([0, 1, 0]) * force
                    for i, force in enumerate(force_centrifugal_i_array)
                ]
            ),
        ),
        axis=0,
    )[0]
    print(f"-------Ma:{np.sum(moment_aero,axis=0)[0]:.0f}Nm----------")
    print(
        f"Mc_equal: {mc_equal:.0f}Nm, Mc_moment:{mc_moment:.0f}Nm, Mc_ft:{mc_ft:.0f}Nm"
    )
    print(f"Fc_equal: {fc_equal:.0f}N, Fc_moment:{fc_moment:.0f}N, Fc_ft:{fc_ft:.0f}N")
    # TODO: this is just oposing each y force directly, essentially just setting it to zero
    # for i,fa_i in enumerate(force_aero):
    #     force_aero[i][1] = 0

    # TODO: remove, additional lines were for testing the calculation
    # calculating the moment due to the centrifugal forces
    # moment_centrifugal_x = np.array([-force_centrifugal_i_array[i]*point[2] for i,point in enumerate(points)])
    # moment_centrifugal_y = np.array([0*point[0] for i,point in enumerate(points)])
    # moment_centrifugal_z = np.array([force_centrifugal_i_array[i]*point[0] for i,point in enumerate(points)])
    force_centrifugal = np.array(
        [
            np.array([0, 1, 0]) * force
            for i, force in enumerate(force_centrifugal_i_array)
        ]
    )
    moment_centrifugal = np.cross(points, force_centrifugal)
    # moment_centrifugal = np.array([np.cross(point,force_centrifugal[i]) for i,point in enumerate(points)])
    # Mc_x = np.array(force_centrifugal[:, 2] * points[:, 1] - force_centrifugal[:, 1] * points[:, 2])
    # Mc_y = np.array(force_centrifugal[:, 0] * points[:, 2] - force_centrifugal[:, 2] * points[:, 0])
    # Mc_z = np.array(force_centrifugal[:, 1] * points[:, 0] - force_centrifugal[:, 0] * points[:, 1])

    # TODO: unphysical correction, which also doesn't work
    # force_centrifugal = solver_utils.optimize_fc_to_get_mx_to_zero(force_centrifugal,points)

    # STRUCTURAL
    # TODO: commented below is initial attempt at bending-resistance, needed for multiple chord-wise points
    # if config.kite_name == 'V9_60C':
    #     force_strut = np.zeros(config.kite.points_ini.shape)
    #     for key in points_between_dict.keys():
    #         point = points[int(key)]
    #         line_end1 = points[points_between_dict[key][0]]
    #         line_end2 = points[points_between_dict[key][1]]
    #         distance,distance_vector,angle = structural_model.distance_point_to_line(point,[line_end1,line_end2])
    #         force_strut[int(key)] += STIFFNESS_DATA['STRUT']*distance_vector        # adding force to force_aero, as an external force
    #     # adding strut force to aero force
    #     force_aero += force_strut

    # EXTERNAL
    force_external = np.zeros(config.kite.points_ini.shape)
    moment_external = np.zeros(config.kite.points_ini.shape)
    for i in range(config.kite.points_ini.shape[0]):
        force_external[i] = force_aero[i] + force_centrifugal[i]
        moment_external[i] = moment_aero[i] + moment_centrifugal[i]

    # SOLVER (here the structural solver is called)
    points_new, solver_max_error = solver_main.calculate_points_new(
        points,
        bridle_rest_lengths,
        wing_rest_lengths,
        force_aero,
        input_structural_solver,
    )

    # TODO: RELAXATION APPLIED TO POINT DISPLACEMENT
    points_new = 0.75 * points_new + 0.25 * points
    # TODO: A type of damping applied after x number of iterations
    if it > 150 and it % 10 == 0:
        config.kite_config.stiffness.bridle["BRIDLE"] *= 1.05
        config.kite_config.stiffness.bridle["TUBE"] *= 1.05
        config.kite_config.stiffness.bridle["TE"] *= 1.05

    # CALCULATE NEW APPARANT WIND SPEED
    # calculate new apparant wind speed distribution
    # TODO: include this effect
    # vel_app_array, r_0, v_k_i_array, center_of_gravity = calculate_vel_app_distribution(force_aero,config.kite.mass_points,vel_app,points_new)

    # Convergence
    # iter_difference with last iteration
    iter_diff = np.amax(np.linalg.norm(points_new - points, axis=1))
    iter_diff_r_0 = np.abs(r_0_new - r_0)
    sideangle_leading_edge_midpoint = (
        functions_print.calculate_side_angle_leading_edge_midpoint(
            points_new[
                config.kite.connectivity.plate_point_indices[
                    int(config.kite.n_segments / 2)
                ]
            ][:2]
        )
    )

    if (
        iter_diff < tol_points_turning
        and solver_max_error < tol_solver_max_error
        and iter_diff_r_0 < tol_r_0
    ):  # if converged
        print(f"Converged at: i{it}, final update of vel_kite_x")
        tol_vk_optimization = 1e-1
        vel_app[0] = -solver_utils.optimalisation_of_vk_for_fx_0(
            -vel_app[0],
            vel_app,
            points,
            config,
            input_VSM,
            input_bridle_aero,
            tol_vk_optimization,
        )
        points = points_new
        is_convergence = True
        break

    # elif it == config.aero_structural.max_iter: #if reached max-iterations
    #     is_convergence = False
    #     break

    # w hen changes are bigger than 2meters
    # or when sideangle > 5deg

    elif (
        iter_diff > max_points_diff_turning
        or np.abs(sideangle_leading_edge_midpoint) > max_sideangle
    ):
        print(
            f"iter_diff: {iter_diff:.3f} > {max_points_diff_turning} and/or {np.abs(sideangle_leading_edge_midpoint)}>{max_sideangle:.3f}deg"
        )
        print(f"--> Change is too large to be reasonable, stopping iterations")
        is_convergence = False
        break

    else:  # if not convergenced
        points = points_new
        r_0 = r_0_new
        if config.is_print_mid_results:
            if it % it_check_turning == 0:
                trim_angle = functions_print.calculate_trim_angle(
                    coord_refined,
                    config.aero.n_splits,
                    points_new[
                        config.kite.connectivity.plate_point_indices[
                            int(config.kite.n_segments / 2)
                        ]
                    ][:2],
                )
                print(
                    f"i{it} - diff: {1e3*iter_diff:.3f}mm - diff r_0: {1e3*iter_diff_r_0:.3f}mm- error: {solver_max_error:.3f}N? - side: {sideangle_leading_edge_midpoint:.2f}deg - trim: {trim_angle:.3f}deg - dt: {(time.time() - time_iter):.2f}s (aero: {(time_iter_aero - time_iter):.2f} - struc: {(time.time() - time_iter_aero):.2f})"
                )

    # TODO: remove
    # , Fc: {np.sum(force_centrifugal_i_array):.2f}N, Fa,y: {np.sum(force_aero_pre_fc[:,1]):.2f}N')
    print(
        f"r_0: {r_0_new:.2f}m, Diff Fy: {np.sum(force_centrifugal_i_array) - (-np.sum(force_aero[:,1])) :.3f}N, vel_app:{vel_app}"
    )
    print(
        f"Mx(roll) : {np.sum(moment_external[:,0]):.1f}, Fx: {np.sum(force_external[:,0]):.1f}, Ma_x: {np.sum(moment_aero[:,0]):.1f}, Mc_x: {np.sum(moment_centrifugal[:,0]):.1f}, Fa_x: {np.sum(force_aero[:,0]):.1f}, Fc_x: {np.sum(force_centrifugal[:,0]):.1f}"
    )
    print(
        f"My(pitch): {np.sum(moment_external[:,1]):.1f}, Fy: {np.sum(force_external[:,1]):.1f}, Ma_y: {np.sum(moment_aero[:,1]):.1f}, Mc_y: {np.sum(moment_centrifugal[:,1]):.1f}, Fa_y: {np.sum(force_aero[:,1]):.1f}, Fc_y: {np.sum(force_centrifugal[:,1]):.1f}"
    )
    print(
        f"Mz(yaw)  : {np.sum(moment_external[:,2]):.1f}, Fz: {np.sum(force_external[:,2]):.1f}, Ma_z: {np.sum(moment_aero[:,2]):.1f}, Mc_z: {np.sum(moment_centrifugal[:,2]):.1f}, Fa_z: {np.sum(force_aero[:,2]):.1f}, Fc_z: {np.sum(force_centrifugal[:,2]):.1f}"
    )

np.save(
    f"results/V3_25/points_up_{str(int(100*config.u_p))}_circular_asymmetric.npy",
    points_new,
)

# post-processing
aero_structural_total_time = time.time() - aerostructural_start_time

### What should be present in a non-jupyter notebook setup


In [ ]:
# %% PLOTTING OUTPUT SEPARATELY
# plotting turning output separately

points_us_10 = np.load(
    f"results/V3_25/points_up_{str(int(100*config.u_p))}_turning_us_10.npy"
)
points_new = points_us_10


# %% what should be included according to python standards


def main():
    print("This is the main function, need to add the right functionalities in here")


# To show that this file is a script and not a module to be imported elsewhere
if __name__ == "__main__":  # "if we run the module directly
    main()  # run the code within this conditional"